In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
import math
import copy

import skimage.io as io
from scipy.misc import bytescale

In [4]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import merge

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled)


# WARNING : this code is still far from being complete

## Build model architecture

### Conditional Random Fields as Recurrent Neural Networks
#### Torr Vision Group
##### Shuai Zheng, Sadeep Jayasumana, Bernardino Romera-Paredes, Vibhav Vineet, Zhizhong Su
##### Dalong Du, Chang Huang, Philip H. S. Torr

http://www.robots.ox.ac.uk/~szheng/papers/CRFasRNN.pdf

### WARNING :
#### In v1 of this script we will only implement the FCN-8s subcomponent of the CRF-RNN network

#### Quotes from MatConvNet page (http://www.vlfeat.org/matconvnet/pretrained/#semantic-segmentation) :
*These networks are trained on the PASCAL VOC 2011 training and (in part) validation data, using Berekely's extended annotations, as well as Microsoft COCO.*

*While the CRF component is missing (it may come later to MatConvNet), this model still outperforms the FCN-8s network above, partially because it is trained with additional data from COCO.*

*The model was obtained by first fine-tuning the plain FCN-32s network (without the CRF-RNN part) on COCO data, then building built an FCN-8s network with the learnt weights, and finally training the CRF-RNN network end-to-end using VOC 2012 training data only. The model available here is the FCN-8s part of this network (without CRF-RNN, while trained with 10 iterations CRF-RNN).*

In [5]:
def convblock(cdim, nb, bits=3):
    L = []
    
    for k in range(1,bits+1):
        convname = 'conv'+str(nb)+'_'+str(k)
        if False:
            # first version I tried
            L.append( ZeroPadding2D((1, 1)) )
            L.append( Convolution2D(cdim, 3, 3, activation='relu', name=convname) )
        else:
            L.append( Convolution2D(cdim, 3, 3, border_mode='same', activation='relu', name=convname) )
    
    L.append( MaxPooling2D((2, 2), strides=(2, 2)) )
    
    return L

In [6]:
def fcn32_blank():
    
    withDO = False # no effect during evaluation but usefull for fine-tuning
    
    if True:
        mdl = Sequential()
        
        # First layer is a dummy-permutation = Identity to specify input shape
        #mdl.add( Permute((1,2,3), input_shape=(3,224,224)) ) # WARNING : 0 is the sample dim
        mdl.add( Permute((1,2,3), input_shape=(3,512,512)) ) # WARNING : 0 is the sample dim

        for l in convblock(64, 1, bits=2):
            mdl.add(l)

        for l in convblock(128, 2, bits=2):
            mdl.add(l)
        
        for l in convblock(256, 3, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 4, bits=3):
            mdl.add(l)
            
        for l in convblock(512, 5, bits=3):
            mdl.add(l)
        
        mdl.add( Convolution2D(4096, 7, 7, border_mode='same', activation='relu', name='fc6') ) # WARNING border
        if withDO:
            mdl.add( Dropout(0.5) )
        mdl.add( Convolution2D(4096, 1, 1, border_mode='same', activation='relu', name='fc7') ) # WARNING border
        if withDO:
            mdl.add( Dropout(0.5) )
        
        # WARNING : model decapitation i.e. remove the classifier step of VGG16 (usually named fc8)
        
        mdl.add( Convolution2D(21, 1, 1,
                               border_mode='same', # WARNING : zero or same ? does not matter for 1x1
                               activation='relu', name='score_fr') )
        
        mdl.add( Deconvolution2D(21, 4, 4, # WARNING : exact meaning ?
                                 output_shape=(None, 21, 34, 34), # WARNING : exact meaning ?
                                 subsample=(2, 2), # WARNING : exact meaning ?
                                 border_mode='valid', # WARNING : valid, same or full ?
                                 activation=None,
                                 name = 'score2') )
        
        mdl.add( Cropping2D(cropping=((1, 1), (1, 1))) ) # WARNING : cropping as deconv gained pixels
        
        return mdl
    
    else:
        # See following link for a version based on Keras functional API :
        # gist.github.com/EncodeTS/6bbe8cb8bebad7a672f0d872561782d9
        raise ValueError('not implemented')

In [7]:
# WARNING : explanation about Deconvolution2D layer
# http://stackoverflow.com/questions/39018767/deconvolution2d-layer-in-keras
# the code example in the help (??Deconvolution2D) is very usefull too

#?? Deconvolution2D

In [8]:
fcn32model = fcn32_blank()

In [9]:
#fcn32model.summary() # visual inspection of model architecture

In [10]:
sp4 = Convolution2D(21, 1, 1,
                    border_mode='same', # WARNING : zero or same ? does not matter for 1x1
                    activation=None, # WARNING : to check
                    name='score_pool4')

# INFO : to replicate MatConvNet.DAGN.Sum layer see documentation at :
# https://keras.io/getting-started/sequential-model-guide/
summed = merge([sp4(fcn32model.layers[14].output), fcn32model.layers[-1].output], mode='sum')

# INFO : final 16x16 upsampling of "summed" using deconv layer upsample_new (32, 32, 21, 21)
upnew = Deconvolution2D(21, 32, 32, # WARNING : exact meaning ?
                        output_shape=(None, 21, 528, 528), # WARNING : exact meaning ?
                        border_mode='valid', # WARNING : valid, same or full ?
                        subsample=(16, 16),
                        activation=None,
                        name = 'upsample_new')

crop8 = Cropping2D(cropping=((8, 8), (8, 8))) # WARNING : cropping as deconv gained pixels

fcn16model = Model(fcn32model.input, crop8(upnew(summed)))

In [11]:
# WARNING : check dim weights against .mat file to check deconvolution setting
upnew.get_weights()[0].shape

(21, 21, 32, 32)

In [12]:
fcn16model.summary() # visual inspection of model architecture

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
permute_input_1 (InputLayer)     (None, 3, 512, 512)   0                                            
____________________________________________________________________________________________________
permute_1 (Permute)              (None, 3, 512, 512)   0           permute_input_1[0][0]            
____________________________________________________________________________________________________
conv1_1 (Convolution2D)          (None, 64, 512, 512)  1792        permute_1[0][0]                  
____________________________________________________________________________________________________
conv1_2 (Convolution2D)          (None, 64, 512, 512)  36928       conv1_1[0][0]                    
___________________________________________________________________________________________

## Load VGG weigths from .mat file

#### https://www.vlfeat.org/matconvnet/pretrained/#semantic-segmentation
##### Download from console with :
wget http://www.vlfeat.org/matconvnet/models/pascal-fcn8s-tvg-dag.mat

In [13]:
from scipy.io import loadmat

In [14]:
data = loadmat('pascal-fcn8s-tvg-dag.mat', matlab_compatible=False, struct_as_record=False)
l = data['layers']
p = data['params']
description = data['meta'][0,0].classes[0,0].description

In [18]:
l.shape, p.shape, description.shape

((1, 46), (1, 40), (1, 21))

In [19]:
class2index = {}
for i, clname in enumerate(description[0,:]):
    class2index[str(clname[0])] = i
    
print(sorted(class2index.keys()))

['aeroplane', 'background', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']


In [20]:
if False: # inspection of data structure
    print(dir(l[0,31].block[0,0]))
    print(dir(l[0,36].block[0,0]))

In [21]:
for i in range(0, p.shape[1]-1, 2):
    print(i,
          str(p[0,i].name[0]), p[0,i].value.shape,
          str(p[0,i+1].name[0]), p[0,i+1].value.shape)

(0, 'conv1_1f', (3, 3, 3, 64), 'conv1_1b', (64, 1))
(2, 'conv1_2f', (3, 3, 64, 64), 'conv1_2b', (64, 1))
(4, 'conv2_1f', (3, 3, 64, 128), 'conv2_1b', (128, 1))
(6, 'conv2_2f', (3, 3, 128, 128), 'conv2_2b', (128, 1))
(8, 'conv3_1f', (3, 3, 128, 256), 'conv3_1b', (256, 1))
(10, 'conv3_2f', (3, 3, 256, 256), 'conv3_2b', (256, 1))
(12, 'conv3_3f', (3, 3, 256, 256), 'conv3_3b', (256, 1))
(14, 'conv4_1f', (3, 3, 256, 512), 'conv4_1b', (512, 1))
(16, 'conv4_2f', (3, 3, 512, 512), 'conv4_2b', (512, 1))
(18, 'conv4_3f', (3, 3, 512, 512), 'conv4_3b', (512, 1))
(20, 'conv5_1f', (3, 3, 512, 512), 'conv5_1b', (512, 1))
(22, 'conv5_2f', (3, 3, 512, 512), 'conv5_2b', (512, 1))
(24, 'conv5_3f', (3, 3, 512, 512), 'conv5_3b', (512, 1))
(26, 'fc6f', (7, 7, 512, 4096), 'fc6b', (4096, 1))
(28, 'fc7f', (1, 1, 4096, 4096), 'fc7b', (4096, 1))
(30, 'score_frf', (1, 1, 4096, 21), 'score_frb', (21, 1))
(32, 'score2f', (4, 4, 21, 21), 'score2b', (21, 1))
(34, 'score_pool4f', (1, 1, 512, 21), 'score_pool4b', (21, 

In [22]:
for i in range(l.shape[1]):
    print(i,
          str(l[0,i].name[0]), str(l[0,i].type[0]),
          [str(n[0]) for n in l[0,i].inputs[0,:]],
          [str(n[0]) for n in l[0,i].outputs[0,:]])

(0, 'conv1_1', 'dagnn.Conv', ['data'], ['conv1_1'])
(1, 'relu1_1', 'dagnn.ReLU', ['conv1_1'], ['conv1_1x'])
(2, 'conv1_2', 'dagnn.Conv', ['conv1_1x'], ['conv1_2'])
(3, 'relu1_2', 'dagnn.ReLU', ['conv1_2'], ['conv1_2x'])
(4, 'pool1', 'dagnn.Pooling', ['conv1_2x'], ['pool1'])
(5, 'conv2_1', 'dagnn.Conv', ['pool1'], ['conv2_1'])
(6, 'relu2_1', 'dagnn.ReLU', ['conv2_1'], ['conv2_1x'])
(7, 'conv2_2', 'dagnn.Conv', ['conv2_1x'], ['conv2_2'])
(8, 'relu2_2', 'dagnn.ReLU', ['conv2_2'], ['conv2_2x'])
(9, 'pool2', 'dagnn.Pooling', ['conv2_2x'], ['pool2'])
(10, 'conv3_1', 'dagnn.Conv', ['pool2'], ['conv3_1'])
(11, 'relu3_1', 'dagnn.ReLU', ['conv3_1'], ['conv3_1x'])
(12, 'conv3_2', 'dagnn.Conv', ['conv3_1x'], ['conv3_2'])
(13, 'relu3_2', 'dagnn.ReLU', ['conv3_2'], ['conv3_2x'])
(14, 'conv3_3', 'dagnn.Conv', ['conv3_2x'], ['conv3_3'])
(15, 'relu3_3', 'dagnn.ReLU', ['conv3_3'], ['conv3_3x'])
(16, 'pool3', 'dagnn.Pooling', ['conv3_3x'], ['pool3'])
(17, 'conv4_1', 'dagnn.Conv', ['pool3'], ['conv4_1'])


In [44]:
# documentation for the dagnn.Crop layer :
# https://github.com/vlfeat/matconvnet/blob/master/matlab/%2Bdagnn/Crop.m

In [24]:
def copy_mat_to_keras(kmodel):
    
    raise ValueError('need to be adapted')
    
    kerasnames = [lr.name for lr in kmodel.layers]

    prmt = (3,2,0,1) # WARNING : important setting as 2 of the 4 axis have same size dimension
    
    for i in range(0, p.shape[1]-1, 2):
        matname = '_'.join(p[0,i].name[0].split('_')[0:-1])
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            print 'found : ', (str(matname), kindex)
            l_weights = p[0,i].value
            l_bias = p[0,i+1].value
            f_l_weights = l_weights.transpose(prmt)
            f_l_weights = np.flip(f_l_weights, 2)
            f_l_weights = np.flip(f_l_weights, 3)
            assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
            assert (l_bias.shape[1] == 1)
            assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
            assert (len(kmodel.layers[kindex].get_weights()) == 2)
            kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])
        else:
            print 'not found : ', str(matname)

In [25]:
#copy_mat_to_keras(fcn32model)
copy_mat_to_keras(fcn16model)

ValueError: need to be adapted